### MTH8211 - Lab 2

Par: Guillaume Thibault - 1948612

<br><br/>

Implementation de la factorisation $$ A = LDL* $$ d'une matric carrée hermitienne

* L : triangulaire inférieur unitaire (diagonale est composée que de 1)
* D est diagonale

In [15]:
using Distributions, LinearAlgebra
using SparseArrays

In [9]:
# Hermitian matrix: complex square matrix that is equal to its own conjugate transpose
A = Matrix(Hermitian(rand(3,3) + I))
@assert ishermitian(A)
display(A)

3×3 Matrix{Float64}:
 1.71087   0.101429  0.509012
 0.101429  1.67452   0.562713
 0.509012  0.562713  1.40175

#### Partie 1.
Si A est définie posotive, on peut garantir l'exsitence de la factorisation, cela est équivalent à la factorisation de Cholesky

1. Dans ce cas de figure, expliquer pourquoi 𝐷 sera également définie positive 

<br> <br/>

    Reponse


2. Algorithme pour a décomposition

    Avec 

    $$ A = \left[ {\begin{array}{cc}  a & b & c \\ b & * & * \\ c & * & * \end{array} } \right] $$

    Nous pouvons trouver la matrice suivante pour enlever l'élément de la première colonne

    $$ L_1 = \left[ {\begin{array}{cc}  1 & 0 & 0 \\ -b/a & 1 & 0 \\ -c/a & 0 & 1 \end{array} } \right] $$

    Comme ça, on peut utiliser $ L_1^T $ pour enlever les éléments de la première ligne.


    Avec $ A_1 = L_1 A L_2^T $, on applique la même procédure pour trouver $ A_2 = L_2 A_1 L_2^T = (L_2 L_1) A (L_2^T L_1^T) $. On continue ainsi jusqu'à la dernière ligne/colonne et on obtient $ D = A_{n-1} = (L_{n-1} ... L_1) A_0 (L_{n-1}^T ... L_1^T) $.

    Enfin, en fixant $ L = (L_{n-1}^T ... L_1^T)^{-1} $ on trouve la factorisation : $A=LDL^T$.

    

3. Read the documentation on $Symmetric$ and $Hermitian$

4. Implementation of the algo

In [10]:
A = [1 0 2+2im 0 3-3im; 0 4 0 5 0; 6-6im 0 7 0 8+8im; 0 9 0 1 0; 2+2im 0 3-3im 0 4];
A = Hermitian(A)

5×5 Hermitian{Complex{Int64}, Matrix{Complex{Int64}}}:
 1+0im  0+0im  2+2im  0+0im  3-3im
 0+0im  4+0im  0+0im  5+0im  0+0im
 2-2im  0+0im  7+0im  0+0im  8+8im
 0+0im  5+0im  0+0im  1+0im  0+0im
 3+3im  0+0im  8-8im  0+0im  4+0im

In [57]:
function ldl(A)
    D = copy(A)
    L  = zeros(size(A)) + I
    m, n = size(A)
    for i in 1:m
        # Lₙ matrix for this it
        Lₙ = zeros(size(A)) + I

        # Find the elements that will cancel the row
        for j in i+1:n
            # Populate Lₙ
            Lₙ[j, i] = - D[j, i] / D[i, i]
            # Fill L
            L[j, i] = - Lₙ[j, i]
        end
        # Find matrix for next iteration Lₙ D Lₙᵀ 
        D = Lₙ * D * Lₙ'
    end
    return L, D
end

ldl (generic function with 1 method)

In [66]:
# Sample case
A = [1 2 2; 2 1 2; 2 2 1];
A = Symmetric(A); # triangle supérieur
display(A);

L, D = ldl(A);

display(L * D * L');

3×3 Symmetric{Int64, Matrix{Int64}}:
 1  2  2
 2  1  2
 2  2  1

3×3 Matrix{Float64}:
 1.0  2.0  2.0
 2.0  1.0  2.0
 2.0  2.0  1.0

In [71]:
# Using triangular sup matrix 
# A = Symmetric(tril(A' * A), :L); # triangle inférieur
A = rand(5, 5);
A = Symmetric(triu(A' * A));
display(A);
L, D = ldl(A);
display(L * D * L');


5×5 Symmetric{Float64, Matrix{Float64}}:
 2.33957  1.36591   2.11782  1.0656    1.424
 1.36591  1.01627   1.36782  0.724459  0.936727
 2.11782  1.36782   2.33414  1.02533   1.22831
 1.0656   0.724459  1.02533  0.620863  0.75731
 1.424    0.936727  1.22831  0.75731   1.00359

5×5 Matrix{Float64}:
 2.33957  1.36591   2.11782  1.0656    1.424
 1.36591  1.01627   1.36782  0.724459  0.936727
 2.11782  1.36782   2.33414  1.02533   1.22831
 1.0656   0.724459  1.02533  0.620863  0.75731
 1.424    0.936727  1.22831  0.75731   1.00359

In [73]:
A = rand(5, 5);
A = Symmetric(tril(A' * A), :L); # triangle inférieur
display(A);
L, D = ldl(A);
display(L * D * L');

5×5 Symmetric{Float64, Matrix{Float64}}:
 1.20408   1.36096   0.51908   1.31658   0.781784
 1.36096   1.9062    0.821708  1.69749   1.17583
 0.51908   0.821708  0.854454  1.14531   0.760853
 1.31658   1.69749   1.14531   2.32492   0.981834
 0.781784  1.17583   0.760853  0.981834  1.04611

5×5 Matrix{Float64}:
 1.20408   1.36096   0.51908   1.31658   0.781784
 1.36096   1.9062    0.821708  1.69749   1.17583
 0.51908   0.821708  0.854454  1.14531   0.760853
 1.31658   1.69749   1.14531   2.32492   0.981834
 0.781784  1.17583   0.760853  0.981834  1.04611

In [67]:
A = [1 0 2+2im 0 3-3im; 0 4 0 5 0; 6-6im 0 7 0 8+8im; 0 9 0 1 0; 2+2im 0 3-3im 0 4];
A = Hermitian(A);
display(A);

L, D = ldl(A);

display(L * D * L');

5×5 Hermitian{Complex{Int64}, Matrix{Complex{Int64}}}:
 1+0im  0+0im  2+2im  0+0im  3-3im
 0+0im  4+0im  0+0im  5+0im  0+0im
 2-2im  0+0im  7+0im  0+0im  8+8im
 0+0im  5+0im  0+0im  1+0im  0+0im
 3+3im  0+0im  8-8im  0+0im  4+0im

InexactError: InexactError: Float64(-2.0 + 2.0im)

In [18]:
using Distributions

μ = 10
σ = 2

m = Normal(μ, σ)
rand(m)

8.13514888312351

ErrorException: syntax: missing comma or ) in argument list